In [ ]:
import requests# …①
import json# …②
import time# …③

In [ ]:
#必要な情報の書き換え
client_id='L××××××××××××××××××××××××××××dlB'
client_secret='8Utc_×××××××××××××××××××××××××××××××××××xzxL_2haMqO2Pw'
domain="https://×××××××.auth0.com"
facebook_user_id="facebook|2××××××××××××××12"
insta_id="17××××××××××××××88"
hash_tag_name='任意のハッシュタグ名を入力'


In [ ]:
#Auth0　api_tokenの取得　

headers = {
    'content-type': 'application/json',
}

data = '{\"client_id\":\"'+client_id+'\",\"client_secret\":\"'+client_secret+'\",\"audience\":\"'+domein+"/api/v2/"+'\",\"grant_type\":\"client_credentials\"}'

response = requests.post(domein+'/oauth/token', headers=headers, data=data)
auth0_api_token=response.json()

print(auth0_api_token['access_token'])

In [ ]:
#facebook 全ての情報を取得　　
#必要な情報　　ユーザー特定のためのFB　USER_ID
#userが特定できていれば、その後、繰り返し取得可能

api_token_auth=auth0_api_token['access_token']

headers = {
     'Authorization': 'Bearer '+str(api_token_auth)
    }
response = requests.get(domein+'/api/v2/users/'+facebook_user_id, headers=headers)
print(response)
userinfo=response.json()

fb_access_token= userinfo['identities'][0]['access_token']
print('fb_access_token:'+fb_access_token)

In [ ]:
userinfo

In [ ]:
#ハッシュタグidを取得

url = 'https://graph.facebook.com/v3.0/ig_hashtag_search/?user_id='+insta_id+'&q='+hash_tag_name+'&access_token='+fb_access_token
r= requests.get(url)
if(r.status_code is not requests.codes.ok):
    print( 'ステータスコード：'+str(r.status_code))
    print('200じゃないので終了')
else:
    json_dict = json.loads(r.text)
    print(json_dict)
    hash_tag_id=json_dict['data'][0]['id']
    print(hash_tag_id)

In [ ]:
##人気の投稿１００件まで取得
#アクセス制限がかかっている場合は、1時間に２０0回までしかコールできない為、60件程度しか取得できない場合もある
#１時間おくとまた取得できた

In [ ]:
contents_list=[]
paging_next=[]
url = 'https://graph.facebook.com/v3.2/'+hash_tag_id+'/top_media/?user_id='+insta_id+'&fields=id,media_type,caption,comments_count,like_count,media_url,children{media_url},permalink&limit=50&access_token='+fb_access_token
r = requests.get(url)
json_dict = json.loads(r.text)
paging_next.append(json_dict['paging']['next'])
contents_list.append(json_dict)
print(json_dict['paging']['next'])
#print(json_dict['data'])

while(json_dict['paging']['next']):
    next_hash=str(json_dict['paging']['next'])
    r = requests.get(next_hash)
    if(r.status_code is not requests.codes.ok):
        print( 'ステータスコード：'+str(r.status_code))
        print('200じゃないので終了')
        break
    else:
        json_dict = json.loads(r.text)
        if(json_dict['data']==[]):
            print('dataが　空なので[アクセスレート制限の可能性高い]break')
            break
        else:
            contents_list.append(json_dict)
            print('コンテンツリストの合計'+str(len(contents_list)))
            #print(json_dict['data'])
            print(json_dict['paging']['next'])
            #1時間に２０0回までしかコールできない  50 200=10000  #実際2500件までしかAPIで取得できない　しかし１時間おくとまた取得できた 
            time.sleep(1)
            num=int(0)
            for json_dict_test in enumerate(contents_list):
                json_dict_num=len(json_dict_test[1]['data'])
                print(json_dict_num)
                num=num+json_dict_num
            print('合計'+str(num))
            if(int(num)>=100):
                print('100以上なので、抜ける')
                break
else:
    print("ループ処理は完了しました。")
    

In [ ]:
len(contents_list)

In [ ]:
len(contents_list[0]['data'])

In [ ]:
single_json_content_list=[]
for json_content in contents_list:
    for single_json_content in json_content['data']:
        print(single_json_content)
        single_json_content_list.append(single_json_content)

In [ ]:
#シングルコンテンツの件数
print(len(single_json_content_list))

In [ ]:
single_json_content_list[0]

In [ ]:
single_json_content_list[0]['id']

In [ ]:
single_json_content_list[0]['caption']

In [ ]:
################################################################
#取得したデータに、ネガポジをつけ、json形式で保存
################################################################

In [ ]:
from pprint import pprint
#str型（文字列）をdict型に変換
import ast

In [ ]:
#キーを取得する
subscription_key = "01896db540db440693630483908e49bc"
assert subscription_key
#APIのエンドポイントの基盤を入力する
#endpoint "https://japaneast.api.cognitive.microsoft.com/text/analytics/v2.0"
text_analytics_base_url = "https://japaneast.api.cognitive.microsoft.com/text/analytics/v2.0/"

In [ ]:
#感情分析を行うAPIのエンドポイントをセット
sentiment_api_url = text_analytics_base_url + "sentiment"
print(sentiment_api_url)

In [ ]:
single_json_content_list[0]

In [ ]:
def change_dict_key(d, old_key, new_key, default_value=None):
    d[new_key] = d.pop(old_key, default_value)

In [ ]:
#APIで感情分析する対象のテキストをjson形式で作成

for json_content in contents_list:
    for single_json_content in json_content['data']:
        single_json_content['language']= 'ja'
        

In [ ]:
#APIで感情分析する対象のテキストをjson形式で作成
azure_json=''
for single_json_for_azure in single_json_content_list:
    try:
        azure_json_single=str({'id': single_json_for_azure['id'],'language': 'ja','text':single_json_for_azure['caption'],'media_type':single_json_for_azure['media_type'],'comments_count':single_json_for_azure['comments_count'],'like_count':single_json_for_azure['like_count'],'permalink':single_json_for_azure['permalink'],'media_url':single_json_for_azure['media_url']})
        azure_json +=azure_json_single+','
    except KeyError:
        azure_json_single=str({'id': single_json_for_azure['id'],'language': 'ja','text':single_json_for_azure['caption'],'media_type':single_json_for_azure['media_type'],'comments_count':single_json_for_azure['comments_count'],'like_count':single_json_for_azure['like_count'],'permalink':single_json_for_azure['permalink'],'children':single_json_for_azure['children']})
        azure_json +=azure_json_single+','
# # # # #最後だけカンマ抜く
azure_json=azure_json[:-1]
azure_json="{'documents' : ["+ azure_json+"]}"

In [ ]:
print(azure_json)
type(azure_json)

In [ ]:
#str型をdictに変換
azure_json=ast.literal_eval(azure_json)
type(azure_json)

In [ ]:
#ドキュメントのセンチメント スコアは 0 〜1 の間であり、高いスコアは肯定的なセンチメントを示します。
headers   = {"Ocp-Apim-Subscription-Key": subscription_key}
response  = requests.post(sentiment_api_url, headers=headers, json=azure_json)
sentiments = response.json()
pprint(sentiments)

In [ ]:
#念のため、ID順に、azure_json['documents']とsentiments['documents']をソートする
sentiments['documents'].sort(key=lambda x: x['id'], reverse=False)
azure_json['documents'].sort(key=lambda x: x['id'], reverse=False)

In [ ]:
#先頭に格納されているデータとid　を確認
print(sentiments['documents'][0])
print(azure_json['documents'][0])

In [ ]:
#最後尾に格納されているデータとid　を確認
print(sentiments['documents'][-1])
print(azure_json['documents'][-1])

In [ ]:
print(len(sentiments['documents']))
print(len(azure_json['documents']))

In [ ]:
sentiments['documents'][50]

In [ ]:
azure_json['documents'][50]

In [ ]:
for num,sentiments_single in enumerate(sentiments['documents']):
    #print(azure_json['documents'][num])
    azure_json['documents'][num]['score']=sentiments['documents'][num]['score']

In [ ]:
#############ポジティブ順に並び替える（Pythonで辞書のリストを特定のキーの値に従ってソート）　###################################

In [ ]:
#辞書をセット
naga_sort=azure_json['documents']
posi_sort=azure_json['documents']

In [ ]:
posi_sort.sort(key=lambda x: x['score'], reverse=True)
posi_sort

In [ ]:
print(posi_sort[0]['text'])
print(posi_sort[0][ 'score'])

In [ ]:
print(posi_sort[-1]['text'])
print(posi_sort[-1][ 'score'])

In [ ]:
#############ネガティブ順に並び替える（Pythonで辞書のリストを特定のキーの値に従ってソート）　###################################

In [ ]:
naga_sort.sort(key=lambda x: x['score'], reverse=False)
naga_sort

In [ ]:
print(naga_sort[0]['text'])
print(naga_sort[0][ 'score'])

In [ ]:
print(naga_sort[-1]['text'])
print(naga_sort[-1][ 'score'])